# Boolean Retrieval Model — Implementasi Lengkap
Notebook ini mengimplementasikan model Boolean Retrieval lengkap: preprocessing teks, incidence matrix, inverted index, evaluator query boolean (AND/OR/NOT + parentheses), dan contoh uji berdasarkan materi serta dataset di folder `data`. Komentar setiap sel bersifat singkat.

In [1]:
# Import dasar dan inisialisasi
import os, re
import pandas as pd

# NLTK: gunakan jika tersedia. Jika resource tidak ada, fungsi akan mencoba tetap bekerja.
try:
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    from nltk.stem import PorterStemmer
    _nltk_available = True
except Exception as e:
    print('NLTK tidak sepenuhnya tersedia:', e)
    _nltk_available = False

print("Ready. Current working dir:", os.getcwd())

Ready. Current working dir: /content


In [2]:
# Siapkan resource NLTK bila memungkinkan (punkt & stopwords)
if _nltk_available:
    try:
        nltk.data.find('tokenizers/punkt')
    except:
        try:
            nltk.download('punkt', quiet=True)
        except:
            pass
    try:
        nltk.data.find('corpora/stopwords')
    except:
        try:
            nltk.download('stopwords', quiet=True)
        except:
            pass
    try:
        stop_words = set(stopwords.words('english'))
    except:
        stop_words = set()
    stemmer = PorterStemmer()
else:
    # fallback minimal stopwords & simple tokenizer
    stop_words = {'the','and','is','in','to','of','a','for','on','with','that','this'}
    stemmer = None

print('Stopwords size (approx):', len(stop_words))

Stopwords size (approx): 198


In [ ]:
# Fungsi preprocessing: lowercase, hapus non-letters, tokenisasi sederhana, stopword removal, stemming (opsional)
def simple_tokenize(text):
    # fallback tokenization: split on whitespace after cleaning
    return [t for t in text.split() if t]

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)    # hanya huruf dan spasi
    # Tokenize: pakai word_tokenize jika tersedia, jika tidak gunakan simple_tokenize
    if _nltk_available:
        tokens = word_tokenize(text)
    else:
        tokens = simple_tokenize(text)
    # Hilangkan stopwords dan token pendek
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]
    # Stemming jika tersedia
    if stemmer is not None:
        try:
            tokens = [stemmer.stem(t) for t in tokens]
        except:
            pass
    return tokens

# Contoh singkat
sample = """Brutus and Caesar! Calpurnia warned, but Caesar ignored."""
print('Sample tokens:', preprocess(sample))

In [ ]:
# Baca semua file .txt di folder 'data' (relatif ke working dir).
data_folder = 'data'  # ubah jika folder berbeda

documents = {}
if os.path.isdir(data_folder):
    for fname in sorted(os.listdir(data_folder)):
        if fname.lower().endswith('.txt'):
            path = os.path.join(data_folder, fname)
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    text = f.read()
            except:
                # fallback baca dengan latin-1
                with open(path, 'r', encoding='latin-1') as f:
                    text = f.read()
            documents[fname] = preprocess(text)
else:
    print(f"Folder '{data_folder}' tidak ditemukan. Pastikan kamu meng-upload folder 'data' di working directory.")

print('Dokumen terbaca:', len(documents))
# tampilkan contoh 3 dokumen pertama (nama dan token)
list(documents.items())[:3]

In [ ]:
# Bangun vocabulary dan incidence matrix (biner)
vocab = sorted(set(term for tokens in documents.values() for term in tokens))
incidence = pd.DataFrame(0, index=vocab, columns=list(documents.keys()))

for doc, tokens in documents.items():
    for t in set(tokens):   # gunakan set -> biner
        incidence.at[t, doc] = 1

print('Vocabulary size:', len(vocab))
print('Incidence matrix shape:', incidence.shape)
# tampilkan sebagian
incidence.head(12)

In [ ]:
# Bangun inverted index: term -> list dokumen
inverted_index = {term: list(incidence.columns[incidence.loc[term] == 1]) for term in incidence.index}
# tampilkan 10 pertama
for term in list(inverted_index.keys())[:10]:
    print(term, '->', inverted_index[term])

In [ ]:
# Evaluator boolean: menerima query string dan incidence matrix (pandas DataFrame)
# Mendukung AND, OR, NOT, dan parenthesis.
def boolean_retrieval(query, incidence_matrix):
    # normalize parentheses spacing and lower
    q = query.lower().replace('(', ' ( ').replace(')', ' ) ')
    tokens = q.split()
    # replace term tokens with binary vectors
    vecs = []
    for t in tokens:
        if t in {'and','or','not','(',')'}:
            vecs.append(t)
        else:
            if t in incidence_matrix.index:
                vecs.append(list(incidence_matrix.loc[t].astype(int)))
            else:
                # term tidak ada => vektor nol
                vecs.append([0]*incidence_matrix.shape[1])

    # helper ops
    def NOT(v): return [int(not x) for x in v]
    def AND(a,b): return [x & y for x,y in zip(a,b)]
    def OR(a,b): return [x | y for x,y in zip(a,b)]

    # Evaluate with recursive parsing for parentheses
    def eval_tokens(items):
        # first handle parentheses by recursion
        out = []
        i = 0
        while i < len(items):
            if items[i] == '(':
                # find matching )
                depth = 1
                j = i+1
                while j < len(items) and depth>0:
                    if items[j] == '(':
                        depth += 1
                    elif items[j] == ')':
                        depth -= 1
                    j += 1
                sub = items[i+1:j-1+1]  # slice of inside tokens
                out.append(eval_tokens(sub))
                i = j
            else:
                out.append(items[i])
                i += 1

        # then handle NOT (unary), left-to-right for this simplified evaluator
        i = 0
        res = []
        while i < len(out):
            if out[i] == 'not':
                # apply NOT to next item
                operand = out[i+1]
                res.append(NOT(operand))
                i += 2
            else:
                res.append(out[i])
                i += 1

        # then handle AND (left-to-right)
        i = 0
        res2 = []
        while i < len(res):
            if i+1 < len(res) and res[i+1] == 'and':
                a = res2.pop() if res2 else res[i]
                b = res[i+2]
                res2.append(AND(a,b))
                i += 3
            else:
                res2.append(res[i])
                i += 1

        # then handle OR (left-to-right)
        i = 0
        final = []
        while i < len(res2):
            if i+1 < len(res2) and res2[i+1] == 'or':
                a = final.pop() if final else res2[i]
                b = res2[i+2]
                final.append(OR(a,b))
                i += 3
            else:
                final.append(res2[i])
                i += 1

        # final should have one vector
        if len(final) != 1:
            # if there are stray tokens (e.g., single term), reduce by OR
            v = final[0]
            for item in final[1:]:
                v = OR(v, item)
            return v
        return final[0]

    result_vec = eval_tokens(vecs)
    # map vector to document names
    docs = [doc for doc, bit in zip(incidence_matrix.columns, result_vec) if bit == 1]
    return docs

# contoh penggunaan singkat jika incidence ada
if incidence.shape[0] > 0:
    print('Contoh query (dataset):', boolean_retrieval('(badminton or baseball) and not obama', incidence))

## Uji: Case Study dari Materi (simulasi dokumen kecil)
Kita akan membuat contoh kecil seperti di slide (Brutus, Caesar, Calpurnia) dan uji query `Brutus AND Caesar AND NOT Calpurnia`.

In [ ]:
# Simulasi dokumen kecil sesuai slide
case_docs = {
    'Doc1': 'Antony Brutus Caesar',
    'Doc2': 'Brutus Caesar Calpurnia',
    'Doc3': 'Caesar Hamlet',
    'Doc4': 'Brutus Caesar Hamlet'
}
case_processed = {k: preprocess(v) for k,v in case_docs.items()}
case_vocab = sorted(set(t for tokens in case_processed.values() for t in tokens))
case_incidence = pd.DataFrame(0, index=case_vocab, columns=list(case_processed.keys()))
for d, tokens in case_processed.items():
    for t in set(tokens):
        case_incidence.at[t, d] = 1

print('Case incidence:')
display(case_incidence)
q = 'brutus and caesar and not calpurnia'
print('Query:', q)
print('Dokumen relevan:', boolean_retrieval(q, case_incidence))

## Uji: Query pada dataset `data`
Beberapa contoh query menggunakan dokumen di folder `data`.

In [ ]:
queries = [
    '(badminton or baseball) and not obama',
    '(shinzo or obama) and (lee or badminton)',
    'queen and not modi'
]

for q in queries:
    print('\nQuery:', q)
    print('Result:', boolean_retrieval(q, incidence))

## Kesimpulan singkat
- Notebook ini mengimplementasikan boolean retrieval lengkap (preprocessing -> incidence -> inverted index -> evaluator).  
- Evalutor mendukung AND, OR, NOT dan tanda kurung.  
- Jika ingin peningkatan: invert index struktural (posisi), optimasi dengan bitset/bitarray, dan pengecekan morphological lebih kuat.